In [1]:
# Import packages
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.express as px


In [2]:
#Load data
df = pd.read_csv("data/World-happiness-report-updated_2024.csv", encoding="ISO-8859-1")

#Sort by country and year
df.sort_values(by=["Country name", "year"], inplace=True)

#Visualization factor column
factors = [
    "Log GDP per capita",
    "Social support",
    "Healthy life expectancy at birth",
    "Freedom to make life choices",
    "Generosity",
    "Perceptions of corruption",
    "Positive affect",
    "Negative affect"
]

#Fill in missing values with data from the previous year
df[factors] = df.groupby("Country name")[factors].ffill()

#Extract year list
years = sorted(df["year"].unique())

In [3]:
#Initialize Dash application
app = dash.Dash(__name__)

#Page layout
app.layout = html.Div([
    html.H1("World Happiness Index Dashboard", style={"textAlign": "center"}),
    
    html.P(
    "This is a visualization dashboard for the World Happiness Index.", 
    style={"textAlign": "left", "fontSize": "16px"}
),

    html.P(
    "1. Which regions have higher or lower happiness indices?",
    style={"textAlign": "left", "fontSize": "20px"}
),
    
    html.P(
    "From the global happiness index map from 2005 to 2023, "
    "we can see that North America and Oceania have consistently high happiness scores, while Asia and Africa have relatively low happiness scores.",
    style={"textAlign": "left", "fontSize": "16px"}
),
   
    dcc.Slider(
        id='year-slider',
        min=min(years),
        max=max(years),
        step=1,
        value=min(years),
        marks={str(year): str(year) for year in years},
        tooltip={"placement": "bottom", "always_visible": True}
    ),

    html.Div([
        html.Button("Pause / Play", id="pause-button", n_clicks=0),
        dcc.Store(id='play-state', data=True)
    ], style={"textAlign": "center", "marginTop": "10px"}),

    dcc.Interval(id='interval-component', interval=2000, n_intervals=0),

    dcc.Graph(id="choropleth-map", style={"marginTop": "30px"}),

    html.P(
    "2. What is the relationship between influencing factors and happiness index?",
    style={"textAlign": "left", "fontSize": "20px"}
),

    html.P([
    "In this data, there are 8 factors that affect the happiness index:",
    html.Br(),
    "a. Log GDP per capita: The natural logarithm of the country's GDP per person, "
    "adjusted for differences in cost of living between countries (purchasing power parity)",
    html.Br(),
    "b. Social support: The average level of responses about whether people have friends or family "
    "to rely on in times of trouble (0 for no, 1 for yes)",
    html.Br(),
    "c. Healthy life expectancy: The average number of years a newborn can expect to live in good health, "
    "based on death rates and life expectancy at different ages",
    html.Br(),
    "d. Freedom to make life choices: The average satisfaction level with the freedom people feel they have to make their own life choices",
    html.Br(),
    "e. Generosity: The level of people's willingness to donate to charity, adjusted for GDP per capita",
    html.Br(),
    "f. Perceptions of corruption: The average level of perceived corruption in government and businesses",
    html.Br(),
    "g. Positive affect: The average level of positive emotions experienced by people the previous day",
    html.Br(),
    "h. Negative affect: The average level of negative emotions experienced by people the previous day",
], style={"textAlign": "left", "fontSize": "18px"}),
    
    html.P([
    "By analyzing the scatter plots of each factor against the average happiness score of each country over the years, "
    "as well as the line charts showing the top 10 and bottom 10 countries in happiness rankings each year,",
    html.Br(),    
    "2.1 we can observe that 'Log GDP per capita', 'Social support', 'Healthy life expectancy at birth', 'Freedom to make life choices', and 'Positive affect' are positively correlated with happiness. "
    "countries with higher happiness scores tend to have higher average values in these factors.",     
    html.Br(),
    "2.2 On the other hand, 'Perceptions of corruption' and 'Negative affect' are negatively correlated with happiness, "
    "with countries that have higher happiness scores generally having lower values for these factors.",
    html.Br(),
    "2.3 As for 'Generosity', while it does not show a strong linear relationship with happiness, "
    "countries with higher happiness scores still tend to have slightly higher scores for this factor than countries with lower happiness scores.",   
    html.Br(),
    "2.4 It can also be observed that the countries with the highest average happiness scores are located in Europe, "
    "while those with the lowest scores are in Asia.",
],style={"textAlign": "left", "fontSize": "16px"}),
    
    html.Div([
        html.Label("Select a Factor:"),
        dcc.Dropdown(
            id='shared-factor-dropdown',
            options=[{'label': f, 'value': f} for f in factors],
            value='Social support',
            clearable=False
        )
    ], style={"width": "50%", "margin": "30px auto"}),

    html.Div([
        dcc.Graph(id="scatter-plot", style={"width": "48%", "display": "inline-block"}),
        dcc.Graph(id="top-bottom-line-chart", style={"width": "48%", "display": "inline-block"})
    ]),

    html.P(
    "3. What factors play an important role in countries with high or low happiness index?",
    style={"textAlign": "left", "fontSize": "20px"}
),
    
    html.P(
    "From the bar chart comparing the Top 2 and Bottom 2 countries in the 2023 happiness index, "
    "we can see that 'Healthy life expectancy at birth' is a key factor in happiness scores because the difference is the largest. "
    "Secondly, 'Perceptions of corruption' is a key factor in distinguishing countries with higher happiness scores, "
    "while in countries with lower happiness scores, 'Freedom to make life choices' shows a greater influence.",
    style={"textAlign": "left", "fontSize": "16px"}
),
    
    html.Div([
        html.Label("Compare 2023 Top 2 vs Bottom 2 by Factor:"),
        dcc.Dropdown(
            id='bar-factor-dropdown',
            options=[{'label': f, 'value': f} for f in factors],
            value='Freedom to make life choices',
            clearable=False
        ),
        dcc.Graph(id="bar-chart-2023")
    ], style={"width": "60%", "margin": "40px auto"})
])

In [4]:
#Year autoplay
@app.callback(
    Output("year-slider", "value"),
    Input("interval-component", "n_intervals"),
    State("year-slider", "value"),
    State("play-state", "data"),
)
def update_year(n_intervals, current_year, is_playing):
    if not is_playing:
        return current_year
    idx = years.index(current_year)
    idx = (idx + 1) % len(years)
    return years[idx]

#Play button switch
@app.callback(
    Output("play-state", "data"),
    Input("pause-button", "n_clicks"),
    State("play-state", "data")
)
def toggle_play_pause(n_clicks, current_state):
    if n_clicks == 0:
        return current_state
    return not current_state

#Map Chart
@app.callback(
    Output("choropleth-map", "figure"),
    Input("year-slider", "value")
)
def update_map(year):
    df_filtered = df[df["year"] == year]
    fig = px.choropleth(
        df_filtered,
        locations="Country name",
        locationmode="country names",
        color="Life Ladder",
        hover_name="Country name",
        color_continuous_scale="Viridis",
        title=f"Happiness Index in {year}"
    )
    return fig

In [5]:
#Scatter plot
regions_df = pd.read_csv("data/UN_Region_Categories.csv")  # 包含 Country name 和 Sub-region Name

#Rename columns in regions_df to match df
regions_df = regions_df.rename(columns={"Country or Area": "Country name"})

#Merge two datasets
df = df.merge(regions_df, on="Country name", how="left")

#Use plotly's built-in chromatography
@app.callback(
    Output("scatter-plot", "figure"),
    Input("shared-factor-dropdown", "value")
)
def update_scatter_plot(x_factor):
    #Calculate the average value for each country
    df_avg = df.groupby(["Country name", "Region Name"])[[x_factor, "Life Ladder"]].mean().reset_index()

    #Draw a scatter plot
    fig = px.scatter(
        df_avg,
        x="Life Ladder",
        y=x_factor,
        color="Region Name",  
        hover_name="Country name",
        title=f"Avg Happiness Score vs {x_factor} (All Years)",
        size_max=60,
        color_discrete_sequence=px.colors.qualitative.Set1,  
        labels={"Life Ladder": "Happiness Score", x_factor: x_factor}  
    )

    #Update chart layout
    fig.update_layout(
        xaxis_title="Happiness Score",
        yaxis_title=x_factor,
        plot_bgcolor="white",  
        legend_title="Region Name",
        showlegend=True
    )
    return fig

In [6]:
#Line chart（Top10 vs Bottom10）
@app.callback(
    Output("top-bottom-line-chart", "figure"),
    Input("shared-factor-dropdown", "value")
)
def update_line_chart(factor):
    yearly_data = []
    for year in years:
        df_year = df[df["year"] == year].sort_values("Life Ladder", ascending=False)
        top10 = df_year.head(10)[factor].mean()
        bottom10 = df_year.tail(10)[factor].mean()
        yearly_data.append({"year": year, "Group": "Top 10", "Average": top10})
        yearly_data.append({"year": year, "Group": "Bottom 10", "Average": bottom10})
    df_line = pd.DataFrame(yearly_data)
    fig = px.line(
        df_line, x="year", y="Average", color="Group",
        markers=True,
        title=f"{factor} Trend: Top 10 vs Bottom 10 Countries"
    )
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title=f"Avg {factor}"
    )
    return fig

In [7]:
#Bar chart
@app.callback(
    Output("bar-chart-2023", "figure"),
    Input("bar-factor-dropdown", "value")
)
def update_bar_chart_2023(factor):
    df_2023 = df[df["year"] == 2023].sort_values("Life Ladder", ascending=False)

    top2 = df_2023.head(2).copy().reset_index(drop=True)
    bottom2 = df_2023.tail(2).copy().sort_values("Life Ladder").reset_index(drop=True)

    top2["Legend"] = [f"Top{i+1} - {row['Country name']}" for i, row in top2.iterrows()]
    bottom2["Legend"] = [f"Bottom{i+1} - {row['Country name']}" for i, row in bottom2.iterrows()]

    combined = pd.concat([top2, bottom2])

    fig = px.bar(
        combined,
        x="Country name",
        y=factor,
        color="Legend",
        title=f"{factor} in 2023: Top 2 vs Bottom 2 Happiness Countries",
        hover_data={factor: True, "Life Ladder": True}
    )

    fig.update_layout(
        xaxis_title="Country",
        yaxis_title=factor,
        showlegend=True,      
        legend_title=None    
    )
    return fig

In [9]:
if __name__ == '__main__':
    app.run(debug=True, port=8050)


